# Libraries

In [ ]:
!pip install yfinance --quiet

import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

# Data

For prediction week ahead I need three weeks of previous data. 

In [ ]:
today = '2021-09-01'
today = pd.to_datetime(today,format='%Y-%m-%d')

For data from Tesla I impute weekends with value of Friday.

In [ ]:
data = yf.Ticker('TSLA')
data = data.history(start= today - pd.to_timedelta(25, unit='d'),end = today, interval='1d', rounding=True)
data = data[['Close']].reset_index()

r = pd.date_range(start=data.Date.min(), end=pd.to_datetime(data.Date.max(), format='%Y-%m-%d'))
data=data.groupby('Date').mean().reindex(r)
data.fillna(method='ffill',inplace=True)
data = data.reset_index().rename({'index':'Date'},axis=1)
data = data.loc[data.Date >= today - pd.to_timedelta(21, unit='d')].reset_index(drop=True).rename({'Date':'date'},axis=1)
data

,date,Close
0,2021-08-11,707.82
1,2021-08-12,722.25
2,2021-08-13,717.17
3,2021-08-14,717.17
4,2021-08-15,717.17
5,2021-08-16,686.17
6,2021-08-17,665.71
7,2021-08-18,688.99
8,2021-08-19,673.47
9,2021-08-20,680.26


In [ ]:
def scale_tesla(data):
  return (data - 550)/(900-550)
def scale_tesla_inv(data):
  return data*(900-550) + 550

In [ ]:
data['Close'] = data.Close.apply(scale_tesla)

I need sentiment for the same period

In [ ]:
elon = pd.read_csv('/content/drive/MyDrive/Assignments/Stock price/Models_TS/elon_sent_2021.csv',index_col=0)
elon['date'] = pd.to_datetime(elon.date,format='%Y-%m-%d')
reddit = pd.read_csv('/content/drive/MyDrive/Assignments/Stock price/Models_TS/reddit_sent_2021.csv',index_col=0)
reddit.fillna(0,inplace=True)
reddit['date'] = pd.to_datetime(reddit.date,format='%Y-%m-%d')
news = pd.read_csv('/content/drive/MyDrive/Assignments/Stock price/Models_TS/news_sent_2021.csv',index_col=0)
news['date'] = pd.to_datetime(news.date,format='%d/%m/%Y')

In [ ]:
data = data.merge(elon,on='date').merge(reddit,on='date').merge(news,on='date')
data

,date,Close,sentiment_x,sentiment_y,sentiment
0,2021-08-11,0.450914,0.184579,-0.031287,0.077531
1,2021-08-12,0.492143,-0.135999,-0.051763,0.579854
2,2021-08-13,0.477629,-0.040768,-0.021046,-0.011807
3,2021-08-14,0.477629,0.279770,0.012921,-0.935877
4,2021-08-15,0.477629,0.115295,0.005846,0.022854
5,2021-08-16,0.389057,0.973914,-0.061619,-0.125491
6,2021-08-17,0.330600,0.958394,-0.066600,-0.110067
7,2021-08-18,0.397114,0.201303,-0.025350,-0.183272
8,2021-08-19,0.352771,0.910960,-0.017951,0.208896
9,2021-08-20,0.372171,0.087581,0.119016,-0.045601


Scale sentiment columns

In [ ]:
from sklearn.externals import joblib
# And now to load...

scaler = joblib.load('/content/scaler.save') 

In [ ]:
data.iloc[:,2:] = scaler.transform(data.iloc[:,2:].to_numpy())
data

,date,Close,sentiment_x,sentiment_y,sentiment
0,2021-08-11,0.450914,0.536927,0.307969,0.549160
1,2021-08-12,0.492143,0.352367,0.213689,0.815019
2,2021-08-13,0.477629,0.407192,0.355118,0.501877
3,2021-08-14,0.477629,0.591729,0.511515,0.012804
4,2021-08-15,0.477629,0.497040,0.478935,0.520222
5,2021-08-16,0.389057,0.991356,0.168312,0.441708
6,2021-08-17,0.330600,0.982422,0.145376,0.449872
7,2021-08-18,0.397114,0.546555,0.335302,0.411128
8,2021-08-19,0.352771,0.955113,0.369368,0.618686
9,2021-08-20,0.372171,0.481084,1.000000,0.483991


# Create window for tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(data.iloc[:,1:])

dataset = dataset.window(21, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda w: w.batch(21))

dataset = dataset.map(lambda w: (w))

dataset = dataset.take(1).batch(8).prefetch(1)

# Model

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from keras.layers import Input, Flatten, Dense, Dropout, LSTM, GRU
from tensorflow.keras import callbacks

In [ ]:
model = Sequential([
    Input(shape=(21,4)),
    #Model layers
    GRU(80, return_sequences=True),
    Flatten(),
    Dense(600, activation="relu"),
    Dense(120, activation="relu"),
    Dense(16, activation="relu"),
    Dense(7)
])

model.load_weights('/content/GRU_model_weights.h5')

# Predictions, 5 days ahead

In [ ]:
predictions = model.predict(dataset)
predictions = scale_tesla_inv(predictions)
predictions

array([[734.1827 , 732.02795, 731.67957, 745.40906, 742.0287 , 739.8125 ,
        745.5849 ]], dtype=float32)

In [ ]:
true = yf.Ticker('TSLA')
true = true.history(start= today ,end = today + pd.to_timedelta(8, unit='d'), interval='1d', rounding=True)
true = true[['Close']].reset_index()
r = pd.date_range(start=today, end=today+pd.to_timedelta(6, unit='d'))
true=true.groupby('Date').mean().reindex(r)
true

,Close
2021-09-01,734.09
2021-09-02,732.39
2021-09-03,733.57
2021-09-04,NaN
2021-09-05,NaN
2021-09-06,NaN
2021-09-07,752.92


In [ ]:
true['pred'] = predictions[0,:]
true

,Close,pred
2021-09-01,734.09,734.182678
2021-09-02,732.39,732.027954
2021-09-03,733.57,731.679565
2021-09-04,NaN,745.409058
2021-09-05,NaN,742.028687
2021-09-06,NaN,739.812500
2021-09-07,752.92,745.584900


In [ ]:
predictions_NN = true.dropna().pred.values

# Predictions week ahead, excluding weekends and bankholidays

In [ ]:
predictions_NN

array([734.1827 , 732.02795, 731.67957, 745.5849 ], dtype=float32)